In [7]:
# Machine Learning - Assignment 2
# Section B: Data Pre-processing

import pandas as pd
import numpy as np

# Example: Load your dataset (edit path as needed)
df = pd.read_csv("../data/Spotify_Youtube.csv")

In [8]:
# --- Feature Engineering ---

# 1. Album Song Count
album_song_count = df.groupby('Album_name')['Track_name'].transform('count')
df['Album_Song_Count'] = album_song_count
# ✅ Covered in class: grouping & aggregation (04-Data-Preprocessing)


KeyError: 'Album_name'